# CONSULTA BD

In [ ]:
import pandas as pd
import os

# Detectar entorno para Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# Definir la ruta base para los archivos en Google Drive
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'

# Cargar el DataFrame de hermanos desde la ruta correcta
df_hermanos = pd.read_csv(os.path.join(RUTA_BASE, "base_datos_hermanos.csv"))

df_hermanos[(df_hermanos['genero'] == 'M') & (df_hermanos['privilegio'] == 'ANC')]



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,nombre_completo,genero,privilegio,activo,f_presidente,f_oracion,f_lectura,f_tesoros,f_perlas,f_vida,f_estudio_libro,f_lector_libro,f_discurso_est,f_demos,f_ayudante,observaciones
4,102,Juan Pablo Alfonso,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
13,11,James Cabieles,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
19,13,Cristian Calderón,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
29,58,William Diaz,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN
47,3,Cesar Hernandez,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
49,1,Orlando Hernandez,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
68,9,Ricardo Morales,M,ANC,1,1,1,0,1,1,1,1,1,0,0,0,NaN
82,23,Edwin Quintero,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN
86,27,Gustavo Ramirez,M,ANC,1,1,1,0,1,1,1,1,0,0,1,1,NaN
88,17,Juan Carlos Riaño,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN


# 📥 FASE 1 v5.0: EXTRACTOR WEB ROBUSTO (WOL)

In [2]:
# =============================================================================
#  🕷️ FASE 1 v5.0: EXTRACTOR UNIVERSAL (CAZADOR AUTOMÁTICO)
# =============================================================================
#  MEJORAS:
#  1. Pide AÑO y MES al usuario (ya no hay que editar fechas a mano).
#  2. Calcula automáticamente los lunes de ese mes.
#  3. Mantiene la lógica "Cazador" para encontrar todas las partes (Maestros).
# =============================================================================

import requests
from bs4 import BeautifulSoup
import json
import re
import os
import calendar
from datetime import date

# RUTA BASE (Verifica que sea correcta)
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILE_JSON = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')

def limpiar_texto(t): return t.replace('\u00a0', ' ').strip() if t else ""

def obtener_html(url):
    try:
        r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=20)
        return BeautifulSoup(r.content, 'html.parser') if r.status_code == 200 else None
    except: return None

def generar_urls_lunes(anio, mes):
    """Genera las URLs de WOL para todos los lunes del mes dado"""
    urls = []
    cal = calendar.Calendar(firstweekday=calendar.MONDAY)
    for semana in cal.monthdatescalendar(anio, mes):
        lunes = semana[0]
        # Solo si el lunes pertenece al mes solicitado
        if lunes.month == mes:
            urls.append(f"https://wol.jw.org/es/wol/dt/r4/lp-s/{lunes.year}/{lunes.month}/{lunes.day}")
    return urls

def extractor_universal():
    print("🌍 GENERADOR DE PROGRAMA JW (Fase 1 v5.0)")
    print("=========================================")

    # INTERACCIÓN CON EL USUARIO
    try:
        anio_input = int(input("👉 Ingresa el AÑO (ej. 2026): "))
        mes_input = int(input("👉 Ingresa el MES (1=Ene, 2=Feb...): "))
    except ValueError:
        return print("❌ Error: Debes ingresar números válidos.")

    urls_a_procesar = generar_urls_lunes(anio_input, mes_input)
    print(f"\n   📅 Se detectaron {len(urls_a_procesar)} semanas para procesar.")

    datos = []

    for url in urls_a_procesar:
        print(f"\n   📥 Descargando: {url} ...")
        soup = obtener_html(url)
        if not soup:
            print("      ❌ Error de conexión.")
            continue

        # 1. INFO GENERAL
        elem_titulo = soup.find('h1', id='p1')
        txt_sem = limpiar_texto(elem_titulo.get_text()) if elem_titulo else "Semana desconocida"
        print(f"      🗓️ {txt_sem}")

        # CANCIONES
        canciones = []
        for c in soup.find_all(['p', 'div']): # A veces cambia el tag
             txt = c.get_text()
             if "Canción" in txt or "Cántico" in txt:
                 num = re.search(r'(\d+)', txt)
                 if num: canciones.append(num.group(1))

        # Lógica simple para asignar (Inicio, Medio, Final)
        c_ini = canciones[0] if len(canciones) > 0 else "---"
        c_med = canciones[1] if len(canciones) > 1 else "---"
        c_fin = canciones[2] if len(canciones) > 2 else "---"

        partes = []
        partes.append({"tipo": "Presidente", "titulo": "Presidente de la Reunión"})
        partes.append({"tipo": "Oracion_Inicio", "titulo": f"Canción {c_ini} y oración"})

        # 2. TESOROS
        tesoros_encontrado = False
        for elem in soup.find_all(['p', 'h3']):
            txt = limpiar_texto(elem.get_text())
            if "(10 mins.)" in txt and "Perlas" not in txt and "Consejo" not in txt:
                tit = txt.replace("(10 mins.)", "").strip()
                partes.append({"tipo": "Tesoros", "titulo": tit})
                tesoros_encontrado = True
                break

        if not tesoros_encontrado: partes.append({"tipo": "Tesoros", "titulo": "Discurso de Tesoros"})
        partes.append({"tipo": "Perlas", "titulo": "Busquemos Perlas Escondidas"})

        # LECTURA
        lectura_tit = "Lectura de la Biblia"
        for elem in soup.find_all(['p', 'h3']):
            if "(4 mins.)" in elem.get_text():
                match = re.search(r'\)\s*(.+)', elem.get_text())
                if match: lectura_tit = match.group(1).strip()
                break
        partes.append({"tipo": "Lectura", "titulo": lectura_tit})

        # 3. MAESTROS (Modo Cazador)
        claves = ["Empiece", "Haga revisitas", "Haga discípulos", "Explique", "Discurso"]
        h3s = soup.find_all('h3')
        count_m = 0
        for h3 in h3s:
            txt = limpiar_texto(h3.get_text())
            if "MAESTROS" in txt.upper() or "Estudio" in txt: continue

            if any(k in txt for k in claves):
                # Limpiar texto
                tit_clean = re.split(r'\(\d', txt)[0].strip().strip('.:0123456789')
                partes.append({"tipo": "Maestros", "titulo": tit_clean})
                count_m += 1
                print(f"         ✅ Maestro: {tit_clean}")

        if count_m == 0:
            print("         ⚠️ ALERTA: No se encontraron partes de Maestros. Verifica el HTML.")

        # 4. VIDA CRISTIANA (NVC)
        secc_vida = soup.find('div', id='section4')
        if secc_vida:
            for h3 in secc_vida.find_all('h3'):
                txt = limpiar_texto(h3.get_text())
                if any(x in txt for x in ["VIDA", "Cántico", "Oración", "Estudio", "Conclusión"]): continue
                tit = re.split(r'\(\d', txt)[0].strip()
                partes.append({"tipo": "NVC", "titulo": tit})

        # FIJOS FINALES
        titulo_ebc = "Estudio Bíblico de Congregación"
        # Intento extra de capturar título del libro
        for h3 in soup.find_all('h3'):
            if "Estudio bíblico" in h3.get_text():
                match = re.search(r'\)\s*(.+)', h3.get_text())
                if match: titulo_ebc = match.group(1).strip()

        partes.append({"tipo": "Estudio_Libro", "titulo": titulo_ebc})
        partes.append({"tipo": "Lector_Libro", "titulo": "Lectura del Estudio Bíblico"})
        partes.append({"tipo": "Oracion_Final", "titulo": f"Canción {c_fin} y oración"})

        datos.append({
            "semana": txt_sem,
            "cancion_inicio": c_ini,
            "cancion_medio": c_med,
            "cancion_final": c_fin,
            "partes": partes
        })

    # GUARDAR
    # Modo 'w' sobrescribe el archivo, perfecto para limpiar errores previos
    with open(FILE_JSON, 'w', encoding='utf-8') as f:
        json.dump(datos, f, indent=4, ensure_ascii=False)

    print(f"\n💾 ¡LISTO! JSON generado para {mes_input}/{anio_input}")
    print(f"   Archivo: {os.path.basename(FILE_JSON)}")

if __name__ == "__main__":
    extractor_universal()

🌍 GENERADOR DE PROGRAMA JW (Fase 1 v5.0)
👉 Ingresa el AÑO (ej. 2026): 2026
👉 Ingresa el MES (1=Ene, 2=Feb...): 1

   📅 Se detectaron 4 semanas para procesar.

   📥 Descargando: https://wol.jw.org/es/wol/dt/r4/lp-s/2026/1/5 ...
      🗓️ 5-11 DE ENERO
         ✅ Maestro:  Empiece conversaciones
         ✅ Maestro:  Haga revisitas
         ✅ Maestro:  Discurso

   📥 Descargando: https://wol.jw.org/es/wol/dt/r4/lp-s/2026/1/12 ...
      🗓️ 12-18 DE ENERO
         ✅ Maestro:  Empiece conversaciones
         ✅ Maestro:  Empiece conversaciones
         ✅ Maestro:  Haga revisitas
         ✅ Maestro:  Discurso

   📥 Descargando: https://wol.jw.org/es/wol/dt/r4/lp-s/2026/1/19 ...
      🗓️ 19-25 DE ENERO
         ✅ Maestro:  Empiece conversaciones
         ✅ Maestro:  Haga revisitas
         ✅ Maestro:  Explique sus creencias

   📥 Descargando: https://wol.jw.org/es/wol/dt/r4/lp-s/2026/1/26 ...
      🗓️ 26 DE ENERO A 1 DE FEBRERO
         ✅ Maestro:  Empiece conversaciones
         ✅ Maestro:  Hag

# 🦴 OLD_Fase2_Generador_Excel.py

In [ ]:
# =============================================================================
#  🤖 ASISTENTE DE PROGRAMACIÓN VMC - FASE 2: EXTRACCIÓN HÍBRIDA (Excel + JSON)
# =============================================================================
#  RAMA: fix-fase1
#  CAMBIOS:
#  1. Genera Excel (Legacy) para respaldo.
#  2. Genera JSON (Nuevo) para el motor de asignación automática.
#  3. Soporta Enero 2026 correctamente.
# =============================================================================

import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import calendar
import json
import time
import os

# Detectar entorno
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# CONFIGURACIÓN
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
RUTA_EXCEL = os.path.join(RUTA_BASE, 'Programación VMC_Septiembre-2025-2026.xlsx')
RUTA_JSON = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')

COLUMNAS_ORDENADAS = [
    'Semana', 'Libro', 'Canción Inicial', 'Tesoros de la Biblia',
    'Segunda Canción', 'Tercera Canción',
    'Maestros Título 1', 'Maestros Título 2', 'Maestros Título 3', 'Maestros Título 4',
    'NVC Título 1', 'NVC Título 2', 'NVC Título 3',
    'Info Lectura Biblia', 'Info Estudio Libro'
]

def obtener_fecha_interactiva():
    print("📅 CONFIGURACIÓN DE BÚSQUEDA")
    try:
        anio = int(input("👉 AÑO (ej. 2026): "))
        mes = int(input("👉 MES (1-12): "))
        return anio, mes
    except: return None, None

def generar_urls_dinamicas(anio, mes):
    print(f"\n🔍 Calculando semanas para: {mes}/{anio}...")
    urls = []
    c = calendar.Calendar(firstweekday=calendar.MONDAY)
    for week in c.monthdatescalendar(anio, mes):
        if week[0].month == mes:
            urls.append(f"https://wol.jw.org/es/wol/dt/r4/lp-s/{week[0].year}/{week[0].month:02d}/{week[0].day:02d}")
    return urls

def limpiar_texto_estudio(texto):
    if not texto: return ""
    libros = ['lfb', 'bt', 'lmd', 'lvs', 'cf', 'rr', 'ia', 'jr']
    for l in libros: texto = re.sub(rf"\b{l}(?=[a-zA-Z0-9])", f"{l} ", texto, flags=re.IGNORECASE)
    return texto

def extraer_informacion(url):
    try:
        soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).content, 'html.parser')
        info = {col: "" for col in COLUMNAS_ORDENADAS}

        # Básicos
        info['Semana'] = soup.find('h1', id='p1').get_text(strip=True) if soup.find('h1', id='p1') else "N/A"
        if soup.find('h2', id='p2'): info['Libro'] = soup.find('h2', id='p2').get_text(strip=True)

        # Tesoros (Lógica Refinada S-38: Discurso de 10 mins)
        for elem in soup.find_all(lambda t: t.name in ['h3','p'] and "(10 mins.)" in t.text):
            if "Perlas" not in elem.text:
                info['Tesoros de la Biblia'] = elem.get_text(strip=True).replace("(10 mins.)", "").strip()
                break

        # Canciones
        canciones = [f"Canción {re.search(r'(\d+)', h3.text).group(1)}" for h3 in soup.find_all('h3') if "Canción" in h3.text and re.search(r'(\d+)', h3.text)]
        if len(canciones) > 0: info['Canción Inicial'] = canciones[0]
        if len(canciones) > 1: info['Segunda Canción'] = canciones[1]
        if len(canciones) > 2: info['Tercera Canción'] = canciones[2]

        # Lectura y Estudio
        for p in soup.find_all('p'):
            if "(4 mins.)" in p.text and not info['Info Lectura Biblia']:
                info['Info Lectura Biblia'] = re.search(r'\((?:4|3)\s+mins?.*?\)\s*(.+)', p.text).group(1).strip()
            if "(30 mins.)" in p.text and not info['Info Estudio Libro']:
                info['Info Estudio Libro'] = limpiar_texto_estudio(re.search(r'\(30\s+mins?.*?\)\s*(.+)', p.text).group(1).strip())

        # Secciones Maestros y Vida
        seccion, titulos_maestros, nvc_titulos = None, [], []
        for h in soup.find_all(['h2', 'h3']):
            txt = h.get_text(strip=True)
            if h.name == 'h2':
                if "MEJORES MAESTROS" in txt.upper(): seccion = "MAESTROS"
                elif "VIDA CRISTIANA" in txt.upper(): seccion = "VIDA"
            elif h.name == 'h3' and seccion:
                if any(x in txt for x in ["Canción", "Conclusión", "Artículo"]): continue
                if "Artículo de estudio" in txt: break

                if seccion == "MAESTROS": titulos_maestros.append(txt)
                elif seccion == "VIDA" and "Estudio bíblico" not in txt: nvc_titulos.append(txt)

        for i, t in enumerate(titulos_maestros[:4]): info[f'Maestros Título {i+1}'] = t
        for i, t in enumerate(nvc_titulos[:3]): info[f'NVC Título {i+1}'] = t
        info['NVC Título 3'] = "Estudio bíblico de la congregación" # Default

        return info
    except Exception as e:
        print(f"❌ Error {url}: {e}")
        return None

def guardar_excel_limpio(datos):
    print(f"\n💾 (LEGACY) Guardando Excel: {os.path.basename(RUTA_EXCEL)}...")
    try:
        book = load_workbook(RUTA_EXCEL) if os.path.exists(RUTA_EXCEL) else pd.ExcelWriter(RUTA_EXCEL, engine='openpyxl').book
        if 'Info-reunion' not in book.sheetnames: book.create_sheet('Info-reunion')
        ws = book['Info-reunion']

        if ws.max_row > 1: ws.delete_rows(2, amount=ws.max_row-1) # Limpiar

        for r_idx, row in enumerate(dataframe_to_rows(pd.DataFrame(datos, columns=COLUMNAS_ORDENADAS), index=False, header=False), 1):
            for c_idx, val in enumerate(row, 1): ws.cell(row=r_idx+1, column=c_idx, value=val)

        book.save(RUTA_EXCEL)
        print("✅ Excel actualizado.")
    except Exception as e: print(f"❌ Error Excel: {e}")

# --- AQUÍ ESTÁ LA NUEVA FUNCIÓN JSON ---
def guardar_json_reuniones(datos):
    print(f"\n💾 (NUEVO) Generando JSON para Motor de Asignación...")
    data_estructurada = []

    for d in datos:
        # Aquí aplicamos lógica básica basada en S-38 para definir requisitos
        semana_obj = {
            "semana": d['Semana'],
            "partes": [
                {"tipo": "Presidente", "titulo": "Presidencia", "requiere": "ANC", "tiempo": 0},
                {"tipo": "Oracion_Inicio", "titulo": "Oración", "requiere": "H_BAUTIZADO", "tiempo": 5},
                {"tipo": "Tesoros", "titulo": d['Tesoros de la Biblia'], "requiere": "ANC_SM", "tiempo": 10},
                {"tipo": "Perlas", "titulo": "Perlas Escondidas", "requiere": "ANC_SM", "tiempo": 10},
                {"tipo": "Lectura", "titulo": d['Info Lectura Biblia'], "requiere": "EST_VARON", "tiempo": 4}
            ]
        }

        # Maestros (Dinámico)
        for i in range(1, 5):
            titulo = d.get(f'Maestros Título {i}')
            if titulo:
                # Análisis simple de texto para inferir si es Discurso (Solo varones)
                req = "ESTUDIANTE"
                if "Discurso" in titulo: req = "EST_VARON"
                semana_obj['partes'].append({"tipo": "Maestros", "titulo": titulo, "requiere": req, "tiempo": 5})

        # Vida Cristiana
        for i in range(1, 3):
            titulo = d.get(f'NVC Título {i}')
            if titulo:
                semana_obj['partes'].append({"tipo": "NVC", "titulo": titulo, "requiere": "ANC_SM", "tiempo": 15})

        semana_obj['partes'].extend([
            {"tipo": "Estudio_Libro", "titulo": "Estudio Bíblico", "requiere": "ANC_SM", "tiempo": 30},
            {"tipo": "Lector_Libro", "titulo": "Lectura EBC", "requiere": "H_BAUTIZADO", "tiempo": 0},
            {"tipo": "Oracion_Final", "titulo": "Oración", "requiere": "H_BAUTIZADO", "tiempo": 5}
        ])

        data_estructurada.append(semana_obj)

    with open(RUTA_JSON, 'w', encoding='utf-8') as f:
        json.dump(data_estructurada, f, indent=4, ensure_ascii=False)
    print(f"✅ JSON guardado en: {os.path.basename(RUTA_JSON)}")

# EJECUCIÓN
if __name__ == "__main__":
    anio, mes = obtener_fecha_interactiva()
    if anio:
        datos = [extraer_informacion(url) for url in generar_urls_dinamicas(anio, mes) if extraer_informacion(url)]
        if datos:
            guardar_excel_limpio(datos)      # Mantenemos el puente viejo
            guardar_json_reuniones(datos)    # Construimos el puente nuevo
            print("\n🎉 FASE 1 COMPLETADA (HÍBRIDA).")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📅 CONFIGURACIÓN DE BÚSQUEDA


✍️ FASE 2: registrador_manual_v3_6.py

In [ ]:
# =============================================================================
#  📝 REGISTRADOR MANUAL v3.6 (DIAGNÓSTICO + FLUJO SEGURO)
# =============================================================================
#  MEJORAS:
#  1. Muestra la lista de asignaciones DETECTADAS antes de empezar.
#     (Si no ves 'Empiece conversaciones' ahí, es culpa del JSON).
#  2. Asegura que ENTER sirva para "Saltar a la siguiente" sin cerrar.
# =============================================================================

import pandas as pd
import json
import os
from datetime import datetime, timedelta

# RUTAS
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILE_DB = os.path.join(RUTA_BASE, 'base_datos_hermanos.csv')
FILE_HIST = os.path.join(RUTA_BASE, 'historial_asignaciones.csv')
FILE_JSON = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')
FECHA_BASE_INICIO = datetime(2026, 1, 5)

def buscar_y_seleccionar(mensaje, df_hermanos):
    while True:
        print(f"      ❓ {mensaje}")
        nombre_busq = input("         Escribe Nombre (o ENTER para saltar): ").strip()

        # SI EL USUARIO DA ENTER (VACÍO) -> RETORNAMOS 'None' PERO SEGUIMOS EL FLUJO
        if not nombre_busq: return None, None, None

        candidatos = df_hermanos[df_hermanos['nombre_completo'].str.contains(nombre_busq, case=False, na=False)]

        if candidatos.empty:
            print("         ❌ No encontrado.")
            continue

        lista = candidatos.values.tolist()
        for i, h in enumerate(lista):
            print(f"         [{i}] {h[1]} ({h[2]})")

        sel = input("         👉 Elige #: ")
        if sel.isdigit() and int(sel) < len(lista):
            elegido = lista[int(sel)]
            return elegido[0], elegido[1], elegido[2]
        else:
            print("         ⚠️ Opción inválida.")

def obtener_asignado_actual(df_hist, semana, tipo):
    if df_hist.empty: return None
    filtro = (df_hist['semana_texto'] == semana) & (df_hist['tipo_asignacion'] == tipo)
    resultado = df_hist[filtro]
    if not resultado.empty: return resultado.iloc[-1]['nombre_hermano']
    return None

def registrar_manual_v3_6():
    print("🕵️ REGISTRADOR CON DIAGNÓSTICO (v3.6)")
    print("======================================")

    if not os.path.exists(FILE_DB) or not os.path.exists(FILE_JSON): return print("❌ Faltan archivos.")
    df_db = pd.read_csv(FILE_DB)
    df_hist = pd.read_csv(FILE_HIST) if os.path.exists(FILE_HIST) else pd.DataFrame(columns=['fecha_asignacion'])
    with open(FILE_JSON, 'r', encoding='utf-8') as f: data_reuniones = json.load(f)

    # 1. SELECCIÓN DE SEMANA
    print("\n📅 ELIGE LA SEMANA:")
    for idx, s in enumerate(data_reuniones):
        print(f"   {idx + 1}. {s['semana']}")

    idx_sem = -1
    while True:
        try:
            val = input("\n👉 Número: ")
            idx_sem = int(val) - 1
            if 0 <= idx_sem < len(data_reuniones): break
        except: pass

    semana_data = data_reuniones[idx_sem]
    semana_texto = semana_data['semana']
    partes_raw = semana_data['partes']
    fecha_lunes = (FECHA_BASE_INICIO + timedelta(weeks=idx_sem)).strftime("%Y-%m-%d")

    # 2. FILTRO
    print("\n   1. Solo Estudiantes | 2. Todo el programa")
    modo = input("   👉 Opción: ")
    tipos_interes = ['Lectura', 'Maestros']
    if modo == '2': tipos_interes += ['Presidente', 'Oracion_Inicio', 'Oracion_Final', 'Tesoros', 'Perlas', 'NVC', 'Estudio_Libro', 'Lector_Libro']

    # --- DIAGNÓSTICO PREVIO ---
    partes_a_procesar = [p for p in partes_raw if p['tipo'] in tipos_interes]

    print(f"\n📋 PLAN DE TRABAJO PARA: {semana_texto}")
    print(f"   Se detectaron {len(partes_a_procesar)} partes en el archivo JSON:")
    for i, p in enumerate(partes_a_procesar):
        print(f"   {i+1}. {p.get('titulo', p['tipo'])}")

    print("\n⚠️ IMPORTANTE: Si faltan partes en la lista de arriba,")
    print("   es porque el JSON está incompleto. Deberás correr la FASE 1 de nuevo.")

    input("\n👉 Presiona ENTER para empezar a registrar...")

    nuevos_registros = []

    # 3. BUCLE DE PREGUNTAS
    for parte in partes_a_procesar:
        tipo = parte['tipo']
        titulo = parte.get('titulo', tipo)

        print(f"\n🔹 {titulo}")

        # Memoria Visual
        actual = obtener_asignado_actual(df_hist, semana_texto, tipo)
        prompt = f"¿Quién?"
        if actual:
            print(f"   ✅ YA TIENE ASIGNADO A: {actual}")
            prompt = f"¿Cambiar a {actual}? (ENTER para saltar)"

        # PREGUNTA PRINCIPAL
        id_h, nom_h, gen_h = buscar_y_seleccionar(prompt, df_db)

        if id_h:
            # Eliminar anterior localmente (si hubiera lógica de borrado complejo)
            # Aquí simplemente agregamos el nuevo al lote de "nuevos_registros"
            nuevos_registros.append({
                'fecha_asignacion': fecha_lunes, 'semana_texto': semana_texto,
                'id_hermano': id_h, 'nombre_hermano': nom_h, 'tipo_asignacion': tipo, 'sala': 'Principal'
            })
            print(f"   💾 Registrado para guardar: {nom_h}")

            # PREGUNTA AYUDANTE
            if tipo == 'Maestros' and "Discurso" not in titulo:
                # Revisar ayudante actual
                actual_ayu = obtener_asignado_actual(df_hist, semana_texto, 'Ayudante') # Simplificado
                prompt_ayu = "¿Quién fue su AYUDANTE?"
                if actual_ayu:
                    print(f"   (Ayudante actual detectado en historial: {actual_ayu})")

                id_a, nom_a, _ = buscar_y_seleccionar(prompt_ayu, df_db)
                if id_a:
                    nuevos_registros.append({
                        'fecha_asignacion': fecha_lunes, 'semana_texto': semana_texto,
                        'id_hermano': id_a, 'nombre_hermano': nom_a, 'tipo_asignacion': 'Ayudante', 'sala': 'Principal'
                    })
                    print(f"   💾 Ayudante registrado: {nom_a}")
        else:
            print("   ⏩ Saltando a la siguiente parte...")

    # 4. GUARDADO FINAL
    if nuevos_registros:
        print("\n" + "="*40)
        print(f"💾 Guardando {len(nuevos_registros)} cambios en el historial...")
        df_nuevos = pd.DataFrame(nuevos_registros)
        df_final = pd.concat([df_hist, df_nuevos], ignore_index=True)
        df_final.to_csv(FILE_HIST, index=False, encoding='utf-8')
        print("✅ Historial Actualizado.")
    else:
        print("\n👋 Finalizado sin cambios nuevos.")

if __name__ == "__main__":
    registrar_manual_v3_6()

🕵️ REGISTRADOR CON DIAGNÓSTICO (v3.6)

📅 ELIGE LA SEMANA:
   1. 5-11 DE ENERO
   2. 12-18 DE ENERO
   3. 19-25 DE ENERO
   4. 26 DE ENERO A 1 DE FEBRERO

👉 Número: 1

   1. Solo Estudiantes | 2. Todo el programa
   👉 Opción: 1

📋 PLAN DE TRABAJO PARA: 5-11 DE ENERO
   Se detectaron 4 partes en el archivo JSON:
   1. Is 19:1-12 (th lección 11).
   2.  Empiece conversaciones
   3.  Haga revisitas
   4.  Discurso

⚠️ IMPORTANTE: Si faltan partes en la lista de arriba,
   es porque el JSON está incompleto. Deberás correr la FASE 1 de nuevo.

👉 Presiona ENTER para empezar a registrar...

🔹 Is 19:1-12 (th lección 11).
   ✅ YA TIENE ASIGNADO A: Duvan Torres
      ❓ ¿Cambiar a Duvan Torres? (ENTER para saltar)
         Escribe Nombre (o ENTER para saltar): 
   ⏩ Saltando a la siguiente parte...

🔹  Empiece conversaciones
      ❓ ¿Quién?
         Escribe Nombre (o ENTER para saltar): Dora
         [0] Dora Mendoza (F)
         👉 Elige #: 0
   💾 Registrado para guardar: Dora Mendoza
      ❓ ¿Qui

# ✍️ Fase 2: Registrador Manual v3.6 (Diagnóstico + Flujo Seguro)

In [ ]:
# =============================================================================
#  📝 FASE 2: REGISTRADOR MANUAL v3.6 (CON DIAGNÓSTICO Y MEMORIA)
# =============================================================================
#  CARACTERÍSTICAS:
#  1. Muestra un "Plan de Trabajo" antes de empezar (para ver si el JSON está bien).
#  2. Te muestra quién está asignado actualmente (para evitar duplicados).
#  3. Si das ENTER sin escribir nombre, salta a la siguiente sin borrar nada.
# =============================================================================

import pandas as pd
import json
import os
from datetime import datetime, timedelta

# --- CONFIGURACIÓN DE RUTAS ---
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILE_DB = os.path.join(RUTA_BASE, 'base_datos_hermanos.csv')
FILE_HIST = os.path.join(RUTA_BASE, 'historial_asignaciones.csv')
FILE_JSON = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')

# FECHA BASE (Primer Lunes de Enero 2026)
FECHA_BASE_INICIO = datetime(2026, 1, 5)

def buscar_y_seleccionar(mensaje, df_hermanos):
    while True:
        print(f"      ❓ {mensaje}")
        nombre_busq = input("         Escribe Nombre (o ENTER para saltar): ").strip()

        # SI EL USUARIO DA ENTER (VACÍO) -> RETORNAMOS 'None'
        if not nombre_busq: return None, None, None

        # Búsqueda flexible
        candidatos = df_hermanos[df_hermanos['nombre_completo'].str.contains(nombre_busq, case=False, na=False)]

        if candidatos.empty:
            print("         ❌ No encontrado. Intenta de nuevo.")
            continue

        lista = candidatos.values.tolist()
        for i, h in enumerate(lista):
            # Asumiendo columnas: [0]id, [1]nombre, [2]genero...
            print(f"         [{i}] {h[1]} ({h[2]})")

        sel = input("         👉 Elige #: ")
        if sel.isdigit() and int(sel) < len(lista):
            elegido = lista[int(sel)]
            return elegido[0], elegido[1], elegido[2]
        else:
            print("         ⚠️ Opción inválida.")

def obtener_asignado_actual(df_hist, semana, tipo):
    """Revisa si ya hay alguien asignado en el historial"""
    if df_hist.empty: return None
    filtro = (df_hist['semana_texto'] == semana) & (df_hist['tipo_asignacion'] == tipo)
    resultado = df_hist[filtro]
    if not resultado.empty: return resultado.iloc[-1]['nombre_hermano']
    return None

def registrar_manual_v3_6():
    print("🕵️ REGISTRADOR CON DIAGNÓSTICO (v3.6)")
    print("======================================")

    # 1. CARGA DE ARCHIVOS
    if not os.path.exists(FILE_DB) or not os.path.exists(FILE_JSON):
        return print("❌ Faltan archivos (DB o JSON).")

    df_db = pd.read_csv(FILE_DB)

    if os.path.exists(FILE_HIST):
        df_hist = pd.read_csv(FILE_HIST)
    else:
        df_hist = pd.DataFrame(columns=['fecha_asignacion', 'semana_texto', 'id_hermano', 'nombre_hermano', 'tipo_asignacion', 'sala'])

    with open(FILE_JSON, 'r', encoding='utf-8') as f:
        data_reuniones = json.load(f)

    # 2. SELECCIÓN DE SEMANA
    print("\n📅 ELIGE LA SEMANA:")
    for idx, s in enumerate(data_reuniones):
        print(f"   {idx + 1}. {s['semana']}")

    idx_sem = -1
    while True:
        try:
            val = input("\n👉 Número: ")
            idx_sem = int(val) - 1
            if 0 <= idx_sem < len(data_reuniones): break
        except: pass

    semana_data = data_reuniones[idx_sem]
    semana_texto = semana_data['semana']
    partes_raw = semana_data['partes']

    # Calcular fecha automática
    fecha_lunes = (FECHA_BASE_INICIO + timedelta(weeks=idx_sem)).strftime("%Y-%m-%d")

    # 3. FILTRO DE TAREAS
    print("\n   1. Solo Estudiantes (Lectura + Maestros)")
    print("   2. Todo el programa")
    modo = input("   👉 Opción: ")

    tipos_interes = ['Lectura', 'Maestros']
    if modo == '2':
        tipos_interes += ['Presidente', 'Oracion_Inicio', 'Oracion_Final', 'Tesoros', 'Perlas', 'NVC', 'Estudio_Libro', 'Lector_Libro']

    # --- DIAGNÓSTICO PREVIO ---
    partes_a_procesar = [p for p in partes_raw if p['tipo'] in tipos_interes]

    print(f"\n📋 PLAN DE TRABAJO PARA: {semana_texto}")
    print(f"   Se detectaron {len(partes_a_procesar)} partes asignables:")
    for i, p in enumerate(partes_a_procesar):
        print(f"   {i+1}. {p.get('titulo', p['tipo'])}")

    print("\n⚠️ IMPORTANTE: Si la lista está vacía o incompleta, revisa tu JSON (Fase 1).")
    input("\n👉 Presiona ENTER para empezar...")

    nuevos_registros = []

    # 4. BUCLE DE PREGUNTAS
    for parte in partes_a_procesar:
        tipo = parte['tipo']
        titulo = parte.get('titulo', tipo)

        print(f"\n🔹 {titulo}")

        # Mostrar estado actual
        actual = obtener_asignado_actual(df_hist, semana_texto, tipo)
        prompt = f"¿Quién?"
        if actual:
            print(f"   ✅ ACTUALMENTE: {actual}")
            prompt = f"¿Cambiar a {actual}? (ENTER para saltar)"

        # PREGUNTA PRINCIPAL
        id_h, nom_h, gen_h = buscar_y_seleccionar(prompt, df_db)

        if id_h:
            nuevos_registros.append({
                'fecha_asignacion': fecha_lunes,
                'semana_texto': semana_texto,
                'id_hermano': id_h,
                'nombre_hermano': nom_h,
                'tipo_asignacion': tipo,
                'sala': 'Principal'
            })
            print(f"   💾 Registrado para guardar: {nom_h}")

            # PREGUNTA AYUDANTE (Solo para Maestros que no sean Discurso)
            if tipo == 'Maestros' and "Discurso" not in titulo:
                prompt_ayu = "¿Quién fue el AYUDANTE?"
                # Buscar si ya tenía ayudante
                # (Simplificado: no mostramos ayudante anterior específico aquí, pero podrías agregarlo)

                id_a, nom_a, _ = buscar_y_seleccionar(prompt_ayu, df_db)
                if id_a:
                    nuevos_registros.append({
                        'fecha_asignacion': fecha_lunes,
                        'semana_texto': semana_texto,
                        'id_hermano': id_a,
                        'nombre_hermano': nom_a,
                        'tipo_asignacion': 'Ayudante',
                        'sala': 'Principal'
                    })
                    print(f"   💾 Ayudante registrado: {nom_a}")
        else:
            print("   ⏩ Saltando a la siguiente parte...")

    # 5. GUARDADO FINAL
    if nuevos_registros:
        print("\n" + "="*40)
        df_nuevos = pd.DataFrame(nuevos_registros)
        df_final = pd.concat([df_hist, df_nuevos], ignore_index=True)
        # Eliminar duplicados simples si re-registras lo mismo
        df_final.drop_duplicates(subset=['semana_texto', 'tipo_asignacion', 'id_hermano'], keep='last', inplace=True)

        df_final.to_csv(FILE_HIST, index=False, encoding='utf-8')
        print(f"✅ Historial Actualizado con {len(nuevos_registros)} cambios.")
    else:
        print("\n👋 Finalizado sin cambios nuevos.")

if __name__ == "__main__":
    registrar_manual_v3_6()

🕵️ REGISTRADOR CON DIAGNÓSTICO (v3.6)

📅 ELIGE LA SEMANA:
   1. 5-11 DE ENERO
   2. 12-18 DE ENERO
   3. 19-25 DE ENERO
   4. 26 DE ENERO A 1 DE FEBRERO


# Utilidad 1: Crear archivo de Ausencias


In [ ]:
# =============================================================================
#  🛠️ UTILIDAD: INICIALIZAR AUSENCIAS (CORREGIDO)
# =============================================================================
#  Crea el archivo ausencias.csv manejando la conexión a Google Drive.
# =============================================================================

import pandas as pd
import os
from google.colab import drive # 1. Importamos la librería de conexión

def inicializar_ausencias():
    print("🚑 INICIALIZANDO SISTEMA DE AUSENCIAS")
    print("-------------------------------------")

    # 2. VERIFICACIÓN DE CONEXIÓN (El paso clave que faltaba)
    if not os.path.exists('/content/drive'):
        print("🔌 Conectando a Google Drive...")
        drive.mount('/content/drive')
    else:
        print("✅ Google Drive ya está conectado.")

    # 3. RUTAS
    # Verifica que esta ruta sea la correcta (sin espacios extraños)
    RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
    FILE_AUSENCIAS = os.path.join(RUTA_BASE, 'ausencias.csv')

    # Validar que la carpeta madre exista
    if not os.path.exists(RUTA_BASE):
        print(f"❌ ERROR: No encuentro la carpeta del proyecto en:")
        print(f"   {RUTA_BASE}")
        return

    # 4. CREACIÓN DEL ARCHIVO
    if not os.path.exists(FILE_AUSENCIAS):
        # Columnas estándar para gestionar vacaciones
        cols = ['id_hermano', 'nombre', 'fecha_inicio', 'fecha_fin', 'motivo']

        df = pd.DataFrame(columns=cols)
        df.to_csv(FILE_AUSENCIAS, index=False, encoding='utf-8')

        print(f"\n✅ ¡ÉXITO! Archivo creado: {os.path.basename(FILE_AUSENCIAS)}")
        print("   Ahora puedes registrar las fechas en que los hermanos no estarán.")
    else:
        print(f"\n⚠️ El archivo ya existe en: {os.path.basename(FILE_AUSENCIAS)}")
        print("   No se ha borrado nada.")

if __name__ == "__main__":
    inicializar_ausencias()

🚑 INICIALIZANDO SISTEMA DE AUSENCIAS
-------------------------------------
✅ Google Drive ya está conectado.

✅ ¡ÉXITO! Archivo creado: ausencias.csv
   Ahora puedes registrar las fechas en que los hermanos no estarán.


# Uilidad 2: Agregar hermano nuevo

In [ ]:
# =============================================================================
#  🛠️ GESTIÓN DE HERMANOS: AGREGAR NUEVO (SIN ROMPER IDs)
# =============================================================================

import pandas as pd
import os

RUTA_BD = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/base_datos_hermanos.csv'

def agregar_hermano_manual():
    print("👤 INSCRIPCIÓN DE NUEVO HERMANO")
    print("-------------------------------")

    # 1. Cargar base de datos actual
    if not os.path.exists(RUTA_BD):
        print("❌ Error: No existe la base de datos.")
        return

    df = pd.read_csv(RUTA_BD)

    # 2. Calcular ID Automático (Max ID + 1) para integridad
    # Si la tabla está vacía, empieza en 1. Si no, busca el máximo.
    nuevo_id = 1
    if not df.empty:
        nuevo_id = df['id'].max() + 1

    print(f"🆔 ID Asignado Automáticamente: {nuevo_id}")

    # 3. Pedir datos
    nombre = input("Nombre Completo: ").strip().title()
    genero = input("Género (M/F): ").strip().upper()
    privilegio = input("Privilegio (PUB/SM/ANC): ").strip().upper()

    # Capacidades básicas (Preguntas rápidas)
    print("\n--- Capacidades (S/N) ---")
    lectura = 1 if input("¿Hace Lectura de Biblia? ").upper() == 'S' else 0
    demos = 1 if input("¿Hace Demostraciones/Ayudante? ").upper() == 'S' else 0
    discurso = 1 if input("¿Hace Discursos Estudiantiles? ").upper() == 'S' else 0

    # 4. Crear el diccionario con la estructura EXACTA
    nuevo_hermano = {
        'id': nuevo_id,
        'nombre_completo': nombre,
        'genero': genero,
        'privilegio': privilegio,
        'activo': 1,
        # Banderas
        'f_presidente': 0, 'f_oracion': 0,
        'f_lectura': lectura,
        'f_tesoros': 0, 'f_perlas': 0, 'f_vida': 0,
        'f_estudio_libro': 0, 'f_lector_libro': 0,
        'f_discurso_est': discurso,
        'f_demos': demos,
        'f_ayudante': demos, # Generalmente si hace demos, es ayudante
        'observaciones': "Ingreso manual"
    }

    # 5. Guardar
    # Convertimos el diccionario a DataFrame y lo concatenamos
    nuevo_df = pd.DataFrame([nuevo_hermano])

    # Alineamos columnas (para evitar desorden si faltan claves)
    # Usamos las columnas del df original como molde
    for col in df.columns:
        if col not in nuevo_df.columns:
            nuevo_df[col] = 0 # Rellenar con 0 lo que no preguntamos

    df_final = pd.concat([df, nuevo_df], ignore_index=True)
    df_final.to_csv(RUTA_BD, index=False, encoding='utf-8-sig')

    print(f"\n✅ ¡Listo! {nombre} ha sido inscrito con el ID {nuevo_id}.")

# Para usarlo, simplemente ejecuta:
# agregar_hermano_manual()

# Utilidad 3: MIGRADOR DE HISTORIAL (LEGACY -> SISTEMA NUEVO)

In [8]:
# =============================================================================
#  🕰️ MIGRADOR DE HISTORIAL v3.0 (RUTA FORZADA)
# =============================================================================

import pandas as pd
import os
import re
from datetime import datetime

# --- CONFIGURACIÓN ---
# Usamos la misma ruta que ya sabemos que funciona para el JSON
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'

# PEGA AQUÍ EL NOMBRE EXACTO QUE SALIÓ EN EL DIAGNÓSTICO
NOMBRE_HISTORICO = 'historico_excel_asginaciones.csv'
NOMBRE_DB = 'base_datos_hermanos.csv'
NOMBRE_SALIDA = 'historial_asignaciones.csv'

# --- MAPEO (Tu diccionario de traducción) ---
MAPEO_COLUMNAS = {
    'PRESIDENCIA': 'Presidente',
    'ORACIÓN': 'Oracion_Inicio',
    'TESOROS DE LA BIBLIA': 'Tesoros',
    'BUSQUEMOS PERLAS ESCONDIDAS': 'Perlas',
    'LECTURA DE LA BIBLIA': 'Lectura',
    'SMM ASIG 1 ESTUD': 'Maestros', 'SMM ASIG 1 ACOMP': 'Ayudante',
    'SMM ASIG 2 ESTUD': 'Maestros', 'SMM ASIG 2 ACOMP': 'Ayudante',
    'SMM ASIG 3 ESTUD': 'Maestros', 'SMM ASIG 3 ACOMP': 'Ayudante',
    'SMM ASIG 4 ESTUD': 'Maestros', 'SMM ASIG 4 ACOMP': 'Ayudante',
    'NVC PARTE 1': 'NVC', 'NVC PARTE 2': 'NVC',
    'ESTUDIO LIBRO': 'Estudio_Libro', 'LECTOR LIBRO': 'Lector_Libro',
    'ORACIÓN FINAL': 'Oracion_Final'
}

def buscar_id_hermano(nombre_excel, df_db):
    if pd.isna(nombre_excel) or str(nombre_excel).strip() in ['', 'nan']: return None, None
    nombre_limpio = str(nombre_excel).strip()

    # 1. Exacto
    res = df_db[df_db['nombre_completo'].str.lower() == nombre_limpio.lower()]
    if not res.empty: return res.iloc[0]['id'], res.iloc[0]['nombre_completo']

    # 2. Parcial
    res = df_db[df_db['nombre_completo'].str.contains(nombre_limpio, case=False, regex=False)]
    if not res.empty: return res.iloc[0]['id'], res.iloc[0]['nombre_completo']

    return None, None

def extraer_fecha_lunes(texto_semana, mes_texto):
    try:
        dia = int(re.search(r'(\d+)', str(texto_semana)).group(1))
        mes_map = {'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12, 'ENERO': 1, 'FEBRERO': 2}
        mes_num = mes_map.get(str(mes_texto).upper().strip(), 1)
        anio = 2026 if mes_num < 9 else 2025
        return f"{anio}-{mes_num:02d}-{dia:02d}"
    except:
        return datetime.now().strftime("%Y-%m-%d")

def ejecutar_migracion():
    print("🚀 INICIANDO MIGRACIÓN v3.0")

    # Construir rutas completas
    path_old = os.path.join(RUTA_BASE, NOMBRE_HISTORICO)
    path_db = os.path.join(RUTA_BASE, NOMBRE_DB)
    path_out = os.path.join(RUTA_BASE, NOMBRE_SALIDA)

    # Verificación estricta
    if not os.path.exists(path_old):
        print(f"❌ ERROR: Python NO ve el archivo en: {path_old}")
        print("   Por favor ejecuta el script de Diagnóstico primero.")
        return

    print(f"   ✅ Archivo encontrado: {NOMBRE_HISTORICO}")

    df_old = pd.read_csv(path_old)
    df_db = pd.read_csv(path_db)

    nuevos = []

    for _, fila in df_old.iterrows():
        semana = fila.get('SEMANA', 'Semana X')
        fecha = extraer_fecha_lunes(semana, fila.get('MES', ''))

        for col_orig, tipo_dest in MAPEO_COLUMNAS.items():
            if col_orig in df_old.columns:
                nombre = fila[col_orig]
                id_h, nombre_real = buscar_id_hermano(nombre, df_db)

                if id_h:
                    nuevos.append({
                        'fecha_asignacion': fecha,
                        'semana_texto': semana,
                        'id_hermano': id_h,
                        'nombre_hermano': nombre_real,
                        'tipo_asignacion': tipo_dest,
                        'sala': 'Principal'
                    })

    if nuevos:
        df_migrado = pd.DataFrame(nuevos)
        # Modo 'append' seguro: Si existe historial, lo carga y añade
        if os.path.exists(path_out):
            df_existente = pd.read_csv(path_out)
            df_final = pd.concat([df_existente, df_migrado], ignore_index=True)
        else:
            df_final = df_migrado

        df_final.drop_duplicates(subset=['semana_texto', 'tipo_asignacion', 'nombre_hermano'], inplace=True)
        df_final.to_csv(path_out, index=False, encoding='utf-8')

        print(f"\n💾 ¡MIGRACIÓN EXITOSA!")
        print(f"   Se agregaron {len(nuevos)} registros históricos.")
        print(f"   Ahora el sistema conoce tu historial desde Septiembre.")
    else:
        print("⚠️ No se generaron registros. Revisa los nombres de las columnas en tu CSV.")

if __name__ == "__main__":
    ejecutar_migracion()

🚀 INICIANDO MIGRACIÓN v3.0
   ✅ Archivo encontrado: historico_excel_asginaciones.csv

💾 ¡MIGRACIÓN EXITOSA!
   Se agregaron 238 registros históricos.
   Ahora el sistema conoce tu historial desde Septiembre.


# Previo: Inicializar la Memoria (Historial Asginaciones)

In [ ]:
# =============================================================================
#  🛠️ UTILIDAD: INICIALIZAR HISTORIAL (CON AUTO-CONEXIÓN DRIVE)
# =============================================================================

import pandas as pd
import os
from google.colab import drive # Importamos la librería de Drive

def inicializar_historial():
    # --- PASO 1: VERIFICAR CONEXIÓN A DRIVE ---
    if not os.path.exists('/content/drive'):
        print("🔌 Conectando a Google Drive...")
        drive.mount('/content/drive')
    else:
        print("✅ Google Drive ya está conectado.")

    # --- PASO 2: CONFIGURACIÓN DE RUTA ---
    # Verifica que esta ruta sea EXACTAMENTE la de tu carpeta
    RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
    RUTA_HISTORIAL = os.path.join(RUTA_BASE, 'historial_asignaciones.csv')

    # Verificar si la carpeta existe antes de intentar guardar
    if not os.path.exists(RUTA_BASE):
        print(f"❌ ERROR CRÍTICO: No encuentro la carpeta en Drive.")
        print(f"   Buscaba: {RUTA_BASE}")
        print("   👉 Verifica si el nombre de la carpeta tiene espacios o tildes diferentes.")
        return

    print(f"\n📂 Verificando historial en: {os.path.basename(RUTA_HISTORIAL)}")

    if os.path.exists(RUTA_HISTORIAL):
        print(f"⚠️ El archivo ya existe. No se ha borrado nada por seguridad.")
    else:
        columnas_maestras = [
            'fecha_asignacion', 'semana_texto', 'id_hermano',
            'nombre_hermano', 'tipo_asignacion', 'sala'
        ]

        df_vacio = pd.DataFrame(columns=columnas_maestras)
        df_vacio.to_csv(RUTA_HISTORIAL, index=False, encoding='utf-8')
        print(f"✅ ¡ÉXITO! Archivo de historial creado y listo.")

if __name__ == "__main__":
    inicializar_historial()

🔌 Conectando a Google Drive...
Mounted at /content/drive

📂 Verificando historial en: historial_asignaciones.csv
✅ ¡ÉXITO! Archivo de historial creado y listo.


# 🧠 FASE 3 v2.0: Motor de Asignación Híbrido (Rellenador de Huecos)

In [ ]:
# =============================================================================
#  🤖 FASE 3 v2.0: MOTOR HÍBRIDO (RESPETA MANUAL + RELLENA AUTOMÁTICO)
# =============================================================================
#  LÓGICA MEJORADA:
#  1. Para cada parte de la semana, mira el historial.
#  2. ¿Ya asignaste a alguien manualmente? -> LO RESPETA (No lo cambia).
#  3. ¿Está vacío? -> EL ROBOT BUSCA UN CANDIDATO Y LO ASIGNA.
# =============================================================================

import pandas as pd
import json
import os
from datetime import datetime, timedelta

# --- CONFIGURACIÓN ---
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILE_DB = os.path.join(RUTA_BASE, 'base_datos_hermanos.csv')
FILE_HIST = os.path.join(RUTA_BASE, 'historial_asignaciones.csv')
FILE_JSON_IN = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')
FILE_JSON_OUT = os.path.join(RUTA_BASE, 'programa_completo_asignado.json')

def cargar_datos():
    if not os.path.exists(FILE_DB) or not os.path.exists(FILE_HIST):
        print("❌ Faltan archivos base.")
        return None, None, None

    df_db = pd.read_csv(FILE_DB)
    df_hist = pd.read_csv(FILE_HIST)
    with open(FILE_JSON_IN, 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    return df_db, df_hist, data_json

def obtener_candidato_ideal(tipo_parte, df_db, df_hist, ids_excluidos, filtro_genero=None):
    """Busca al hermano apto que lleva más tiempo sin hacer ESTA asignación"""

    # 1. Filtrar candidatos aptos (Por ahora asumimos todos, luego filtraremos por reglas)
    candidatos = df_db.copy()

    # Filtro Género (Si aplica)
    if filtro_genero:
        candidatos = candidatos[candidatos['genero'] == filtro_genero]

    # 2. Excluir los que ya trabajan esta semana
    candidatos = candidatos[~candidatos['id'].isin(ids_excluidos)]

    if candidatos.empty: return None

    ranking = []
    for _, h in candidatos.iterrows():
        # Buscar última vez que hizo CUALQUIER COSA (Rotación General)
        # Ojo: Para ser más estricto, podrías buscar la última vez que hizo 'tipo_parte'
        hist_h = df_hist[df_hist['id_hermano'] == h['id']]

        if hist_h.empty:
            ultima_fecha = datetime(2020, 1, 1) # Prioridad máxima
        else:
            fechas = pd.to_datetime(hist_h['fecha_asignacion'])
            ultima_fecha = fechas.max()

        ranking.append({
            'id': h['id'],
            'nombre': h['nombre_completo'],
            'ultima_fecha': ultima_fecha
        })

    # Ordenar: El de fecha más antigua primero
    df_rank = pd.DataFrame(ranking).sort_values(by='ultima_fecha', ascending=True)

    if not df_rank.empty:
        return df_rank.iloc[0] # Retornamos el mejor candidato
    return None

def ejecutar_motor_hibrido():
    print("⚙️ INICIANDO MOTOR HÍBRIDO (S-140 v2.0)")
    print("========================================")

    df_db, df_hist, programa_base = cargar_datos()
    if df_db is None: return

    # Asegurar formato fecha
    df_hist['fecha_asignacion'] = pd.to_datetime(df_hist['fecha_asignacion'])

    programa_final = []

    # REGLAS BÁSICAS DE GÉNERO (Ajusta según necesites)
    SOLO_VARONES = ['Presidente', 'Oracion_Inicio', 'Oracion_Final', 'Tesoros', 'Lectura', 'Estudio_Libro', 'Lector_Libro', 'NVC']

    # Recorremos cada semana del JSON (Enero)
    for semana in programa_base:
        semana_texto = semana['semana']
        print(f"\n📅 Procesando: {semana_texto}")

        partes_asignadas = []
        ids_usados_esta_semana = []

        # 1. IDENTIFICAR QUIÉNES YA ESTÁN ASIGNADOS MANUALMENTE (CSV)
        asignaciones_manuales = df_hist[df_hist['semana_texto'] == semana_texto]

        # Pre-llenamos la lista de excluidos con los manuales para no repetirlos
        ids_usados_esta_semana = asignaciones_manuales['id_hermano'].tolist()

        # 2. BARRIDO PARTE POR PARTE
        for parte in semana['partes']:
            tipo = parte['tipo']
            titulo = parte.get('titulo', tipo)

            parte_final = parte.copy()

            # A) VERIFICAR SI YA EXISTE EN HISTORIAL (MANUAL)
            # Filtramos por tipo exacto. OJO: Si hay dos 'Maestros', esto necesita cuidado.
            # Para simplificar, si hay manuales disponibles de ese tipo, consumimos uno.

            candidato_manual = None

            # Buscamos en el pool de asignaciones manuales de esta semana
            matches = asignaciones_manuales[asignaciones_manuales['tipo_asignacion'] == tipo]

            # Truco para manejar múltiples partes del mismo tipo (ej. 2 Maestros):
            # Necesitamos saber cuál de los manuales corresponde a esta parte específica.
            # Como el CSV no guarda el título exacto a veces, usaremos orden FIFO (First In, First Out).

            # Si encontramos registros manuales para este tipo de parte...
            if not matches.empty:
                # Tomamos el primero disponible que no hayamos "marcado" como usado en este loop
                # (Nota: ids_usados ya los tiene todos, pero necesitamos asociar nombre a parte específica)

                # Estrategia simple: Buscar por nombre en el historial si coincide algo del título (opcional)
                # O simplemente tomar el primero de la lista 'matches' y quitarlo de la lista temporal

                # Vamos a usar una lista auxiliar para no asignar al mismo manual dos veces en el loop
                for idx, row in matches.iterrows():
                    # Verificamos si este ID ya fue asignado en *este* ciclo de partes_asignadas
                    ya_asignado_loop = any(p.get('id_asignado') == row['id_hermano'] for p in partes_asignadas)

                    if not ya_asignado_loop:
                        candidato_manual = row
                        break

            if candidato_manual is not None:
                # --- CASO 1: YA TIENE DUEÑO MANUAL ---
                parte_final['asignado_a'] = candidato_manual['nombre_hermano']
                parte_final['id_asignado'] = candidato_manual['id_hermano'] # Meta-dato interno
                parte_final['origen'] = "MANUAL"
                print(f"   ✅ Mantenido Manual: {tipo} -> {candidato_manual['nombre_hermano']}")

                # Caso especial Ayudante: Si la parte tiene ayudante manual
                # (El JSON actual no estructura ayudantes dentro de partes, pero el historial sí tiene tipo 'Ayudante')
                # Por ahora, el JSON plano de Fase 1 no anida ayudantes, así que el PDF lo manejará aparte.

            else:
                # --- CASO 2: ESTÁ VACÍO -> AUTOMATIZAR ---
                print(f"   🤖 Rellenando Hueco: {titulo} ({tipo})...")

                genero = 'M' if tipo in SOLO_VARONES or "Discurso" in titulo else None
                # Si es Discurso de Maestros, forzamos Varón
                if tipo == 'Maestros' and "Discurso" in titulo: genero = 'M'

                candidato = obtener_candidato_ideal(tipo, df_db, df_hist, ids_usados_esta_semana, genero)

                if candidato is not None:
                    parte_final['asignado_a'] = candidato['nombre']
                    parte_final['id_asignado'] = candidato['id']
                    parte_final['origen'] = "AUTO"

                    ids_usados_esta_semana.append(candidato['id'])

                    # ACTUALIZAR HISTORIAL EN MEMORIA (Para que cuente para las siguientes semanas)
                    nueva_fila = {
                        'fecha_asignacion': datetime.now(),
                        'semana_texto': semana_texto,
                        'id_hermano': candidato['id'],
                        'tipo_asignacion': tipo,
                        'nombre_hermano': candidato['nombre']
                    }
                    df_hist = pd.concat([df_hist, pd.DataFrame([nueva_fila])], ignore_index=True)
                    print(f"      ✨ Asignado Auto: {candidato['nombre']}")
                else:
                    parte_final['asignado_a'] = "PENDIENTE"
                    print(f"      ⚠️ No encontré candidato.")

            partes_asignadas.append(parte_final)

        # Guardar la semana procesada
        semana_final = semana.copy()
        semana_final['partes'] = partes_asignadas
        programa_final.append(semana_final)

    # GUARDAR JSON FINAL
    with open(FILE_JSON_OUT, 'w', encoding='utf-8') as f:
        json.dump(programa_final, f, indent=4, ensure_ascii=False)

    print(f"\n✅ PROGRAMA COMPLETADO. Archivo: {os.path.basename(FILE_JSON_OUT)}")

if __name__ == "__main__":
    ejecutar_motor_hibrido()